In [ ]:
import src.log_parser

In [ ]:
!python src/log_parser.py

In [ ]:
from src.log_parser import get_googledb, get_matlabdb, parse_files_for_trial_info, merge_old_new
from oauth2client.service_account import ServiceAccountCredentials

import datetime
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%reload_ext autoreload
%autoreload 2

In [ ]:
# Get the databases
googledb = get_googledb()
matlabdb = get_matlabdb()


In [ ]:
# For 'hard' writes
mergedtbl = pd.merge(matlabdb, googledb, on=['formatted_date', 'animal'])

# Save file
datestr = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
file_to_store = open("logs/explogs.pkl", "wb")
pickle.dump({'googledb': googledb, 'matlabdb': matlabdb, 'last_updated': datestr}, file_to_store)

In [ ]:
# Update the db
file_to_load = open("logs/explogs.pkl", "rb")
data = pickle.load(file_to_load)
file_to_load.close()
matlabdb_old, googledb_old = data['matlabdb'], data['googledb']
   
googledb_merged, googledups = merge_old_new(googledb_old, googledb)
matlabdb_merged, matlabdups = merge_old_new(matlabdb_old, matlabdb, ['comments'])

datestr = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
file_to_store = open("logs/explogs.pkl", "wb")
pickle.dump({'googledb': googledb_merged, 'matlabdb': matlabdb_merged, 'last_updated': datestr}, file_to_store)
file_to_store.close()

In [ ]:
file_to_load = open("logs/explogs.pkl", "rb")
data = pickle.load(file_to_load)
file_to_load.close()
matlabdb, googledb = data['matlabdb'], data['googledb']
   

In [ ]:
import smartload.smartload as smart
import os
data = smart.loadmat('/Users/minhnhatle/Documents/ExternalCode/rigbox_analysis/notebooks/logs/logdb.mat')
vals = data['logtable']['table']['data']
colnames = data['logtable']['columns']

datadict = {}
for colname, val in zip(colnames, vals):
    datadict[colname] = val

matlablog = pd.DataFrame(datadict)
# Parse the experiment log files to find out trial info etc
rigboxdir = '/Users/minhnhatle/Dropbox (MIT)/Nhat/Rigbox'
rootdir = rigboxdir
files = matlablog.value
file = '2021-08-19_1_f23'
date, sessid, animal = file.split('_')
path = os.path.join(rootdir, animal, date, sessid, file + '_Block.mat')
data2 = smart.loadmat(path)

In [ ]:
from src.log_parser import parse_files_for_trial_info

In [ ]:
[elem['rewardDelay'] for elem in data2['block']['paramsValues']]

In [ ]:
ntr, nbl, flags, maxdelays = parse_files_for_trial_info(rigboxdir, files[300:400],verbose=1)

In [ ]:
maxdelays

In [ ]:
from src.log_parser import get_matlabdb

In [ ]:
a = get_matlabdb()

In [ ]:
a.maxdelays.unique()

In [ ]:
a